In [1]:
# 필요한 패키지를 설치합니다
%%capture
!pip install pororo -q
!pip install imapclient -q

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/news_feeder')

In [3]:
import os
os.getcwd()

'/content'

In [18]:
page=1
company_code="005930"

In [20]:
url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page) 
source_code = requests.get(url).text
html = BeautifulSoup(source_code, "lxml")

In [22]:
# 뉴스 링크
links = html.select('.title') 

link_result =[]
for link in links: 
    add = 'https://finance.naver.com' + link.find('a')['href']
    link_result.append(add)

In [ ]:
# 삼성전자가 최고경영자(CEO) 직속으로
# article_html

# <div class="scr01" id="news_read">

In [25]:
article_url = link_result[0] 
article_source_code = requests.get(article_url).text
article_html = BeautifulSoup(article_source_code, "lxml")

In [35]:

# 뉴스 내용
article_contents = article_html.select('.scr01')
article_contents=article_contents[0].get_text()
article_contents = re.sub('\n','',article_contents)

In [ ]:

# 뉴스 제목 
titles = html.select('.title')
title_result=[]
for title in titles: 
    title = title.get_text() 
    title = re.sub('\n','',title)
    title_result.append(title)


# 뉴스 링크
links = html.select('.title') 

link_result =[]
for link in links: 
    add = 'https://finance.naver.com' + link.find('a')['href']
    link_result.append(add)


# 뉴스 날짜 
dates = html.select('.date') 
date_result = [date.get_text() for date in dates] 


# 뉴스 매체     
sources = html.select('.info')
source_result = [source.get_text() for source in sources] 


# 변수들 합쳐서 해당 디렉토리에 csv파일로 저장하기 

result= {"날짜" : date_result, "언론사" : source_result, "기사제목" : title_result, "링크" : link_result} 
print(result)
df_result = pd.DataFrame(result)

In [10]:
""" 주식종목 뉴스(네이버 파이넌스) Crawling 하기 """  
 
 
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os 
 
 
# os.chdir('자신의 디렉토리 주소')
os.chdir('/content/drive/MyDrive/Colab Notebooks/news_feeder')
 
 
def crawler(company_code, maxpage):
    
    # page = 1
    
    # while page <= int(maxpage): 
    for page in range(1, maxpage+1):
        url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page) 
        source_code = requests.get(url).text
        html = BeautifulSoup(source_code, "lxml")
 
        # 뉴스 제목 
        titles = html.select('.title')
        title_result=[]
        for title in titles: 
            title = title.get_text() 
            title = re.sub('\n','',title)
            title_result.append(title)
 
 
        # 뉴스 링크
        links = html.select('.title') 
 
        link_result =[]
        for link in links: 
            add = 'https://finance.naver.com' + link.find('a')['href']
            link_result.append(add)
 
 
        # 뉴스 날짜 
        dates = html.select('.date') 
        date_result = [date.get_text() for date in dates] 
 
 
        # 뉴스 매체     
        sources = html.select('.info')
        source_result = [source.get_text() for source in sources] 
 
 
        # 변수들 합쳐서 해당 디렉토리에 csv파일로 저장하기 
 
        result= {"날짜" : date_result, "언론사" : source_result, "기사제목" : title_result, "링크" : link_result} 
        print(result)
        df_result = pd.DataFrame(result)
        
        print("다운 받고 있습니다------")

    return df_result
        # df_result.to_csv('page' + str(page) + '.csv', mode='w', encoding='utf-8-sig') 



In [11]:
# 종목 리스트 파일 열기  
# 회사명을 종목코드로 변환 
        
def convert_to_code(company, maxpage):
    
    data = pd.read_csv('company_list.txt', dtype=str, sep='\t')   # 종목코드 추출 
    company_name = data['회사명']
    keys = [i for i in company_name]    #데이터프레임에서 리스트로 바꾸기 
 
    company_code = data['종목코드']
    values = [j for j in company_code]
 
    dict_result = dict(zip(keys, values))  # 딕셔너리 형태로 회사이름과 종목코드 묶기 
    
    pattern = '[a-zA-Z가-힣]+' 
    
    if bool(re.match(pattern, company)) == True:         # Input에 이름으로 넣었을 때  
        company_code = dict_result.get(str(company))
        crawler(company_code, maxpage)
 
    
    else:                                                # Input에 종목코드로 넣었을 때       
        company_code = str(company)      
        crawler(company_code, maxpage)
        

In [12]:
convert_to_code("삼성전자",1)

{'날짜': [' 2021.02.21 18:14', ' 2021.02.21 14:15', ' 2021.02.21 18:07', ' 2021.02.21 13:22', ' 2021.02.21 11:44', ' 2021.02.21 10:15', ' 2021.02.21 09:38', ' 2021.02.21 18:01', ' 2021.02.21 14:20', ' 2021.02.21 14:20', ' 2021.02.21 13:27', ' 2021.02.21 12:35', ' 2021.02.21 10:15', ' 2021.02.21 09:44', ' 2021.02.21 09:38', ' 2021.02.21 08:57', ' 2021.02.21 17:57', ' 2021.02.21 17:43', ' 2021.02.21 17:42'], '언론사': ['파이낸셜뉴스', '파이낸셜뉴스', '서울경제', '머니투데이', '아시아경제', '조선비즈', '이데일리', '한국경제', '파이낸셜뉴스', '파이낸셜뉴스', '매일경제', '헤럴드경제', '아시아경제', '머니투데이', '서울경제', '이데일리', '서울경제', '한국경제', '매일경제'], '기사제목': ["삼성 미래사업 '로봇'… CEO직속 TF 꾸렸다", "[단독]삼성전자, 로봇TF 신설 '삼성봇' 상용화 눈앞", '동학개미, 작년 삼성전자 배당금 1조 챙겼다', '삼성전자 몰빵한 개미들, 배당금 1조 받는다', '지난해 개미들 삼성전자 배당금 1兆 돌파…전년比 3배 UP', '삼성전자 투자 개미 배당금 1조원 넘게 받는다', "'동학 개미' 지난해 삼성전자 배당 1.1조원…전년比 3배↑", "삼성디스플레이 'ESG 경영' 강화한다", '삼성디스플레이, ESG 경영 가속화‥RBA 가입', '삼성디스플레이 "ESG 강화한다"…RBA 연합 가입', '삼성디스플레이, RBA 가입…애플·인텔 함께 ESG 경영 박차', '삼성디스플레이, RBA 가입 ‘ESG 경영 본격 시동’', '삼성디스플레이, ESG 경영 본격화…RBA 가입'

ValueError: ignored

In [ ]:

 
 
def main():
    info_main = input("="*50+"\n"+"실시간 뉴스기사 다운받기."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    
    company = input("종목 이름이나 코드 입력: ") 
    
    maxpage = input("최대 뉴스 페이지 수 입력: ")  
 
    convert_to_code(company, maxpage) 
 
 
 
main() 

In [3]:
# 간단하고 깔끔한 요약 Pororo
from pororo import Pororo
summary = Pororo(task="summary", lang="kr")

In [6]:
from send_email import SendEmail

In [7]:
emailAddr = "mattias.colab@gmail.com"   #MK: sender email address
emailPass = "gdgtowpjureldhcc"         #MK: password for sender email


sendMail=SendEmail([emailAddr],emailAddr, emailPass,  "Test Subject", "Test Content")

Success: Email is sent


In [9]:
#"#순이 (요약기)의 이메일. 여기서는 gmail을 사용한다. second gmail을 하나 생성하는 것이 안전하고 좋다.
import getpass
username = 'mattias.colab'
password = getpass.getpass(username + "'s password: ")

mattias.colab's password: ··········


In [10]:
# SSL은 살살 가도록
import ssl
# context manager ensures the session is cleaned up
ssl_context = ssl.create_default_context()

# don't check if certificate hostname doesn't match target hostname
ssl_context.check_hostname = False

# don't check if the certificate is trusted by a certificate authority
ssl_context.verify_mode = ssl.CERT_NONE

In [11]:
# 메일 보내기 함수, gmail의 SMTP를 사용한다.
import smtplib

def sendmail(to, title, body, inreply="new"):
    sum_body =  "TL;DR: " + summary(body)
    message = "Subject: " + title + \
        "\nIn-Reply-To: " + inreply + \
        "\nFrom: Auto Summary <autosumkr@gmail.com>" + \
        "\n\n" + sum_body + \
        "\n---\n" + body

    with smtplib.SMTP_SSL("smtp.gmail.com", context=ssl_context) as server:
        server.login(username, password)
        server.sendmail(username, to, message.encode("utf8"))


In [12]:
# IMAP로 메일을 가져와 요약한다
from imapclient import IMAPClient
import email

def check_imap():
    with IMAPClient(host="imap.gmail.com", ssl_context=ssl_context) as client:
        client.login(username, password)
        client.select_folder('INBOX')

        # search criteria are passed in a straightforward way
        # (nesting is supported)
        messages = client.search(['NOT', 'DELETED'])

        # fetch selectors are passed as a simple list of strings.
        response = client.fetch(messages, ['FLAGS', 'RFC822'])

        # `response` is keyed by message id and contains parsed,
        # converted response items.
        for message_id, data in response.items():
            email_message = email.message_from_bytes(data[b"RFC822"])
            efrom = email_message.get("From")
            eto = email_message.get("To")
            esubject = email_message.get("Subject")
            inreply = email_message.get("Message-ID")

            if eto is None or efrom is None:
              continue

            if eto == username + "@gmail.com":
                mailto = efrom
            else:
                mailto = eto
            
            ebody = ""
            for part in email_message.walk():
                if part.get_content_type() == 'text/plain':
                    charset = part.get_content_charset()
                    ebody = str(part.get_payload(decode=True), 'utf-8', 'ignore')
            
            #print("Sending", mailto, message_id, inreply, esubject)  
            try:          
              sendmail(mailto, esubject, ebody, inreply)
            except Exception as e:
              print(e)
              print("Skipping!", message_id)


            client.delete_messages(message_id)
            print("Sent!", message_id)
        client.expunge()
        client.logout()

In [13]:
check_imap()

name 'summary' is not defined
Skipping! 1
Sent! 1
name 'summary' is not defined
Skipping! 2
Sent! 2
name 'summary' is not defined
Skipping! 3
Sent! 3
name 'summary' is not defined
Skipping! 4
Sent! 4
name 'summary' is not defined
Skipping! 5
Sent! 5
name 'summary' is not defined
Skipping! 6
Sent! 6
name 'summary' is not defined
Skipping! 7
Sent! 7
name 'summary' is not defined
Skipping! 8
Sent! 8
name 'summary' is not defined
Skipping! 9
Sent! 9
name 'summary' is not defined
Skipping! 10
Sent! 10
name 'summary' is not defined
Skipping! 11
Sent! 11
name 'summary' is not defined
Skipping! 13
Sent! 13
name 'summary' is not defined
Skipping! 14
Sent! 14
name 'summary' is not defined
Skipping! 15
Sent! 15
name 'summary' is not defined
Skipping! 16
Sent! 16
name 'summary' is not defined
Skipping! 17
Sent! 17
name 'summary' is not defined
Skipping! 18
Sent! 18
name 'summary' is not defined
Skipping! 19
Sent! 19
name 'summary' is not defined
Skipping! 20
Sent! 20
name 'summary' is not defined


In [24]:
# 이제 1분단위 (60초)로 무한반복 
from time import sleep
import datetime

while(True):
  print("Checking again at", datetime.datetime.now())
  check_imap()
  sleep(60)

Checking again at 2021-02-20 11:48:52.287006


LoginError: ignored

![image](https://i.stack.imgur.com/lkvoo.jpg)

In [ ]:
# https://stackoverflow.com/questions/57113226
while True:pass